<a href="https://colab.research.google.com/github/shivammehta007/NLPResearch/blob/master/QGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Generation

Additional Dependencies

In [0]:
%%capture
!pip install fairseq
!pip install sacremoses subword_nmt
!pip install -U tqdm

In [0]:
import os
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

## DataSet

In [4]:
SQUAD_DIR = '/content/drive/My Drive/Colab Notebooks/SQuAD'
SQUAD_TRAIN = os.path.join(SQUAD_DIR, 'train_v2.json')
# SQUAD_DEV = os.path.join(SQUAD_DIR, 'dev.json')
SQUAD_TEST = os.path.join(SQUAD_DIR, 'test_v2.json')
print(SQUAD_TRAIN, SQUAD_TEST) # , SQUAD_DEV

/content/drive/My Drive/Colab Notebooks/SQuAD/train_v2.json /content/drive/My Drive/Colab Notebooks/SQuAD/test_v2.json


In [0]:
with open(SQUAD_TRAIN) as train_file:
    train_data = json.load(train_file)
    train_data = train_data['data']

with open(SQUAD_TEST) as test_file:
    test_data = json.load(test_file)
    test_data = test_data['data']

### PreProcessing Function

In [0]:
def convert_to_file_without_answers(dataset, dataset_type='train', get_impossible=False):
    para_output = open(dataset_type + '.paragraphs', 'w')
    question_output = open(dataset_type + '.questions', 'w')
    d = []
    for paragraphs in tqdm(dataset):
        paragraphs = paragraphs['paragraphs']
        for i, paragraph in enumerate(paragraphs):
            para = paragraph['context']
            for questionanswers in paragraph['qas']:
                if questionanswers['is_impossible']:
                    continue
                question = questionanswers['question']
                para = para.replace('\n', ' ')
                para_output.write(para.strip() + '\n')
                question_output.write(question.strip() + '\n')
                d.append(i)
    print(len(d))
    para_output.close()
    question_output.close()

In [28]:
convert_to_file_without_answers(train_data, 'train')
convert_to_file_without_answers(test_data, 'test')



86821



5928


In [29]:
!fairseq-preprocess --source-lang paragraphs --target-lang questions \
     --trainpref train --testpref test \
     --destdir train_test_data

Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='train_test_data', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='paragraphs', srcdict=None, target_lang='questions', task='translation', tensorboard_logdir='', testpref='test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='train', user_dir=None, validpref=None, workers=1)
| [paragraphs] Dictionary: 184775 types
| [paragraphs] train.paragraphs: 86821 sents, 10487486 tokens, 0.0% replaced by <unk>
| [paragraphs] Dictionary: 184775 types
| [paragraphs] test.paragraphs: 5928 sents, 767710 tokens, 6.37% re

checkpoints/         test.paragraphs
dict.paragraphs.txt  test.questions
dict.questions.txt   train.paragraphs
drive/               train.paragraphs-questions.paragraphs.bin
fairseq/             train.paragraphs-questions.paragraphs.idx
preprocessed_data/   train.paragraphs-questions.questions.bin
sample_data/         train.paragraphs-questions.questions.idx
set.paragraphs       train.questions
set.questions        train_test_data.paragraphs-questions/


In [34]:
!fairseq-train train_test_data.paragraphs-questions \
     --lr 0.001 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
     --arch fconv_iwslt_de_en --save-dir checkpoints/fconv

Namespace(arch='fconv_iwslt_de_en', best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=0.1, cpu=False, criterion='cross_entropy', curriculum=0, data='train_test_data.paragraphs-questions', dataset_impl=None, ddp_backend='c10d', decoder_attention='True', decoder_embed_dim=256, decoder_embed_path=None, decoder_layers='[(256, 3)] * 3', decoder_out_embed_dim=256, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.2, empty_cache_freq=0, encoder_embed_dim=256, encoder_embed_path=None, encoder_layers='[(256, 3)] * 4', fast_stat_sync=False, find_unused_parameters=False, fix_batches_to_gpus=False, fixed_validation_seed=None, force_anneal=None, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, keep_interval_updates=-1, keep_last_epochs=-1, lazy_load=False, left_pad_source='True', left_pad_target='

[Errno 2] No such file or directory: 'fairseq/examples/translation/'
/content/fairseq


In [43]:
# %cd fairseq/examples/translation/
# !bash prepare-iwslt14.sh
# !fairseq-preprocess --source-lang de --target-lang en \
#     --trainpref examples/translation/iwslt14.tokenized.de-en/train --validpref examples/translation/iwslt14.tokenized.de-en/valid --testpref examples/translation/iwslt14.tokenized.de-en/test \
#     --destdir data-bin/iwslt14.tokenized.de-en
# !mkdir -p checkpoints/fconv
# !fairseq-train data-bin/iwslt14.tokenized.de-en \
#     --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
#     --arch fconv_iwslt_de_en --save-dir checkpoints/fconv

Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/iwslt14.tokenized.de-en', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='de', srcdict=None, target_lang='en', task='translation', tensorboard_logdir='', testpref='examples/translation/iwslt14.tokenized.de-en/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='examples/translation/iwslt14.tokenized.de-en/train', user_dir=None, validpref='examples/translation/iwslt14.tokenized.de-en/valid', workers=1)
| [de] Dictionary: 8847 types
| [de] examples/translation/iwslt14.tokenized.de-en/train.de: 1

In [53]:
# !fairseq-generate data-bin/iwslt14.tokenized.de-en \
#     --path checkpoints/fconv/checkpoint20.pt \
#     --batch-size 128 --beam 5

Streaming output truncated to the last 5000 lines.
P-4254	-0.8423 -0.0697 -0.0231 -1.3360 -0.0947 -0.3391 -1.0005 -0.2375 -0.0338 -0.1083 -0.1547 -0.0655 -0.0575 -0.0035 -0.7784 -0.1169 -0.0490 -1.5823 -0.0173 -0.0000 -1.0876 -0.0852 -1.0187 -0.5899 -1.9508 -1.2880 -0.0001 -0.3518 -0.5043 -0.4493 -0.0535 -0.0008
S-5933	dann sagte sie : » als nächstes ver@@ spr@@ ich mir , dass du immer das richtige tun wirst . « auch wenn es schwer fällt , das richtige zu tun . «
T-5933	then she said , &quot; the second thing i want you to promise me is that you &apos;ll always do the right thing even when the right thing is the hard thing . &quot;
H-5933	-0.48818087577819824	then she said , &quot; next , i jum@@ ped out that you will always do the right thing . &quot;
P-5933	-0.4030 -0.0979 -0.0686 -0.0167 -0.0488 -0.9313 -1.3593 -0.3812 -2.1689 -0.1708 -1.0318 -1.4008 -0.1126 -0.8998 -0.5470 -0.0975 -0.1609 -0.0038 -0.2470 -0.3395 -0.1462 -0.1066
S-413	unsere fähigkeit gene@@ tischen code zu schreibe

dict_keys(['paragraphs', 'title'])